### Task 2

In [130]:
openai_key = ''

In [131]:
import pandas as pd
import os
import datetime
import string
from nltk import FreqDist
from tqdm import tqdm
import spacy
import numpy as np
import openai
from openai.error import ServiceUnavailableError, APIError, Timeout

current_path = os.getcwd()
current_path = current_path.split('Beer_Recommendation_Engine')
path = current_path[0] + 'Beer_Recommendation_Engine'
path

'/Users/drknt_rises/Desktop/Personal/Git Repos/Beer_Recommendation_Engine'

In [132]:
beer_reviews = pd.read_csv(path + '/Data/beer_reviews.csv')
beer_reviews = beer_reviews.drop_duplicates()

def process_date(data):
    from datetime import date

    o_date = str(data)
    o_date = o_date.split(' ')
    if o_date[1] == 'at':
        return date.today()
    else:
        return data


beer_reviews = beer_reviews.loc[~beer_reviews['comment'].isnull()]
beer_reviews['date'] = beer_reviews['date'].apply(lambda row: process_date(row))
beer_reviews['date'] = pd.to_datetime(beer_reviews['date'])
beer_reviews = beer_reviews.loc[beer_reviews['date'] >= datetime.datetime(year=2019, month=1, day=1)]
beer_reviews = beer_reviews.reset_index(drop=True)
beer_reviews = beer_reviews.drop_duplicates('comment')
beer_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11893 entries, 0 to 11963
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   pos           11893 non-null  int64         
 1   beer_name     11893 non-null  object        
 2   final_rating  11893 non-null  float64       
 3   looks         11893 non-null  float64       
 4   smell         11893 non-null  float64       
 5   taste         11893 non-null  float64       
 6   feel          11893 non-null  float64       
 7   overall       11893 non-null  float64       
 8   user          11893 non-null  object        
 9   place         11893 non-null  object        
 10  comment       11893 non-null  object        
 11  date          11893 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(6), int64(1), object(4)
memory usage: 1.2+ MB


In [133]:
def generate_corpus(column, include_stopwords=True, stop_words=[]):
    corpus = " "
    if include_stopwords:
        for text in column:
            text = str(text)
            for i in list(string.punctuation):
                text = text.replace(i, ' ')
            
            text = text.lower()
            corpus += text
        
        corpus = corpus.split()
    else:
        for text in column:
            text = str(text)
            for i in list(string.punctuation):
                text = text.replace(i, ' ')
            
            text = text.lower()
            corpus += text
        
        corpus = corpus.split()

        corpus_filtered = []
        stop_words = set(stop_words)
        for word in corpus:
            if word not in stop_words:
                corpus_filtered.append(word)
        
        corpus = corpus_filtered
    
    return corpus

In [134]:
en = spacy.load("en_core_web_sm")
stop_words = en.Defaults.stop_words
corpus = generate_corpus(beer_reviews['comment'], include_stopwords=False, stop_words=stop_words)

word_freq = FreqDist()
for word in tqdm(corpus):
    word_freq[word] += 1

freq_dist = []
for rank, word in enumerate(word_freq):
    freq_dist.append([rank+1, word, word_freq[word]])

freq_dist = pd.DataFrame(freq_dist, columns=['Rank', 'Word', 'Count'])
freq_dist.to_csv(path + '/Support Files/Freq_Dist_new_comments.csv', index=False)
freq_dist

100%|██████████| 607954/607954 [00:00<00:00, 2605485.02it/s]


,Rank,Word,Count
0,1,beer,8692
1,2,head,6471
2,3,s,6144
3,4,taste,5652
4,5,chocolate,4784
...,...,...,...
22509,22510,to”,1
22510,22511,extend,1
22511,22512,canister,1
22512,22513,8000th,1


In [135]:
def process_column(column):
    data = list(column)
    for i in range(len(data)):
        text = str(data[i])
        for punct in list(string.punctuation):
             text = text.replace(punct, ' ')
        text = text.lower()
        data[i] = text
    return data

def replace_car_brands(data, map):
    for i in range(len(data)):
        text = data[i]
        l_text = text.split()
        for word in l_text:
            if word in map.keys():
                text = text.replace(word, map[word])
        data[i] = text.split()
    return data

In [136]:
comments = process_column(beer_reviews['comment'])

beer_attribute_map = pd.read_csv(path + '/Support FIles/Beeer_attributes.csv')

beer_attribute_dict = {}
for brand, model in zip(beer_attribute_map['Attribute'], beer_attribute_map['Variance']):
    beer_attribute_dict[model] = brand

comments = replace_car_brands(comments, beer_attribute_dict)
print(set(beer_attribute_dict.values()))

{'cinnamon', 'tangerine', 'honey', 'vanilla', 'robust', 'cocoa', 'melon', 'spicy', 'apricot', 'silky', 'crisp', 'sweet', 'chocolaty', 'citrus', 'hoppy', 'fruity', 'papaya', 'estery', 'balanced', 'aggressive', 'roast/toast', 'coffee', 'tart', 'vintage', 'herbal', 'cherry', 'malty', 'dark', 'complex', 'diacetyl'}


In [137]:
attributes = list(set(beer_attribute_dict.values()))

def tf(text, attribute, map):
    text = str(text)
    text = text.strip()
    
    for punct in list(string.punctuation):
        text = text.replace(punct, ' ')
    text = text.lower()
    
    l_text = text.split()
    for word in l_text:
        if word in map.keys():
            text = text.replace(word, map[word])
    
    text = text.split()
    
    count = 0
    for t_word in text:
        if t_word==attribute:
            count += 1
    
    return count

for attribute in (pbar := tqdm(attributes)):
    pbar.set_description(f"Calculating TFs for {attribute}")
    beer_reviews[attribute] = beer_reviews['comment'].apply(lambda row: tf(row, attribute, beer_attribute_dict))
beer_reviews

Calculating TFs for diacetyl: 100%|██████████| 30/30 [00:06<00:00,  4.38it/s]   


,pos,beer_name,final_rating,looks,smell,taste,feel,overall,user,place,...,roast/toast,coffee,tart,vintage,herbal,cherry,malty,dark,complex,diacetyl
0,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,poidog,Hawaii,...,0,0,0,0,0,0,0,0,0,0
1,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,Filabee,Michigan,...,0,0,0,0,0,0,0,0,0,0
2,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,alexnblanton,Arkansas,...,0,0,0,0,0,0,0,0,0,0
3,1,Kentucky Brunch Brand Stout,4.67,4.00,4.50,5.00,4.50,4.50,beergoot,Colorado,...,0,0,0,0,0,0,0,3,0,0
4,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,dcotom,Iowa,...,0,0,0,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11959,250,Samuel Adams Utopias,4.76,4.25,5.00,4.75,4.50,4.75,Robomunky,Washington,...,0,0,0,0,0,0,0,0,0,0
11960,250,Samuel Adams Utopias,4.49,3.75,4.75,4.50,4.25,4.50,chinchill,South Carolina,...,0,0,0,0,0,0,0,0,1,0
11961,250,Samuel Adams Utopias,4.38,4.50,4.00,4.50,4.50,4.50,superspak,North Carolina,...,8,0,0,1,0,4,1,4,4,0
11962,250,Samuel Adams Utopias,4.63,4.00,4.75,4.75,4.50,4.50,skiking70,Maryland,...,0,0,0,0,0,0,0,0,0,0


In [138]:
def idf(column, attribute, map):
    texts = list(column)
    n = len(texts)
    count = 0
    for i in range(n):
        text = texts[i]
        text = str(text)
        text = text.strip()
        
        for punct in list(string.punctuation):
            text = text.replace(punct, ' ')
        text = text.lower()
        
        l_text = text.split()
        for word in l_text:
            if word in map.keys():
                text = text.replace(word, map[word])
        
        text = text.split()

        if attribute in text:
            count += 1
        
    idf = np.log10(n / count)
    return idf

dict_idf = {}
for attribute in (pbar := tqdm(attributes)):
    dict_idf[attribute] = idf(beer_reviews['comment'], attribute, beer_attribute_dict)

dict_idf

100%|██████████| 30/30 [00:06<00:00,  4.55it/s]


{'cinnamon': 1.3949559054687393,
 'tangerine': 1.3217083599903958,
 'honey': 1.61740952214931,
 'vanilla': 0.8114187420180788,
 'robust': 1.8992001598276211,
 'cocoa': 1.4259565601711606,
 'melon': 1.436802161928665,
 'spicy': 1.5687863864784304,
 'apricot': 1.558095520933328,
 'silky': 1.539997298840532,
 'crisp': 1.4191932168704706,
 'sweet': 0.6115476976362433,
 'chocolaty': 0.6457006166600009,
 'citrus': 0.7233019634476703,
 'hoppy': 1.300045159143066,
 'fruity': 0.949160011621318,
 'papaya': 1.5374723238100283,
 'estery': 2.140792967639735,
 'balanced': 0.7641735762207967,
 'aggressive': 1.837245315754507,
 'roast/toast': 1.2031351461350095,
 'coffee': 0.768223468222004,
 'tart': 1.398597809258436,
 'vintage': 1.3380987761785652,
 'herbal': 1.5129985544268276,
 'cherry': 1.5412653128271674,
 'malty': 0.9297842474736399,
 'dark': 0.665358295552008,
 'complex': 1.0284062280455923,
 'diacetyl': 2.6950801771716963}

In [139]:
for attribute in attributes:
    beer_reviews[attribute + '_tf-idf'] = (beer_reviews[attribute] * dict_idf[attribute])
beer_reviews

,pos,beer_name,final_rating,looks,smell,taste,feel,overall,user,place,...,roast/toast_tf-idf,coffee_tf-idf,tart_tf-idf,vintage_tf-idf,herbal_tf-idf,cherry_tf-idf,malty_tf-idf,dark_tf-idf,complex_tf-idf,diacetyl_tf-idf
0,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,poidog,Hawaii,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
1,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,Filabee,Michigan,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
2,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,alexnblanton,Arkansas,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
3,1,Kentucky Brunch Brand Stout,4.67,4.00,4.50,5.00,4.50,4.50,beergoot,Colorado,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,1.996075,0.000000,0.0
4,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,dcotom,Iowa,...,0.000000,0.0,0.0,1.338099,0.0,1.541265,0.929784,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11959,250,Samuel Adams Utopias,4.76,4.25,5.00,4.75,4.50,4.75,Robomunky,Washington,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
11960,250,Samuel Adams Utopias,4.49,3.75,4.75,4.50,4.25,4.50,chinchill,South Carolina,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.028406,0.0
11961,250,Samuel Adams Utopias,4.38,4.50,4.00,4.50,4.50,4.50,superspak,North Carolina,...,9.625081,0.0,0.0,1.338099,0.0,6.165061,0.929784,2.661433,4.113625,0.0
11962,250,Samuel Adams Utopias,4.63,4.00,4.75,4.75,4.50,4.50,skiking70,Maryland,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0


In [140]:
attributes = input(f"Enter three attributes seperated by comma and without space from the list: {set(beer_attribute_dict.values())}")
attributes = attributes.split(',')
passer = attributes.copy()

def similarity(data, vect_2):
    vect_1 = []
    for value in data:
        vect_1.append(value)
    
    sum1 = 0
    sum2 = 0
    sum3 = 0
    for i, j in zip(vect_1, vect_2):
        sum1 = sum1 + (i * j)
        sum2 = sum2 + (i * i)
        sum3 = sum3 + (j * j)
    
    if sum1==0 or sum2==0 or sum==0:
        similarity_score = 0
    else:
        similarity_score = sum1 / (np.sqrt(sum2) * np.sqrt(sum3))
    # print(vect_1, vect_2, similarity_score)
    return similarity_score
    
vect = []
for i, attribute in enumerate(attributes):
    vect.append(dict_idf[attribute])
    attributes[i] = attribute + '_tf-idf'

beer_reviews['similarity'] = beer_reviews[attributes].apply(tuple, axis=1)
beer_reviews['similarity'] = beer_reviews['similarity'].apply(lambda row: similarity(row, vect))
beer_reviews[['beer_name', 'comment', 'similarity']]

,beer_name,comment,similarity
0,Kentucky Brunch Brand Stout,"Amazing stout, expensive but worth the price!",0.00000
1,Kentucky Brunch Brand Stout,This beer meet and exceeded all the unreal hyp...,0.00000
2,Kentucky Brunch Brand Stout,Let it sit and warm to the room a little ... u...,0.00000
3,Kentucky Brunch Brand Stout,"A small pour, ebony dark, no real head. Intens...",0.00000
4,Kentucky Brunch Brand Stout,"Vintage 2022, served on tap at Toppling Goliat...",0.00000
...,...,...,...
11959,Samuel Adams Utopias,It seems weird to rate this like it's a normal...,0.00000
11960,Samuel Adams Utopias,"Shared 2017 canister. Fabulous, very complex a...",0.39424
11961,Samuel Adams Utopias,8000th review! 8 ounce pour from a 24 ounce bo...,0.39424
11962,Samuel Adams Utopias,Such a huge beer and a joy to drink. This is d...,0.00000


In [141]:
re_calculate = False
if re_calculate == True:
    def sentiment_openai(data):
        # openai.api_key = openai_key
        i = 0
        for iter in tqdm(data):
            sents = []
            final_rating, text = iter
            openai.api_key = openai_key
            prompt = f'''Provide a sentiment score for "{text}" in a range of +5 to -5. This comment was overall rated as {final_rating} by the user on a scale of 0-5. Watchout out for the light human sarcasm in the quoted text. Only provide the number.'''

            try:
                # @backoff.on_exception(backoff.expo, (ServiceUnavailableError, APIError))
                def chat_completion(prompt):
                    response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[
                            {"role": "system", "content": prompt},
                        ]
                    )

                    score = response['choices'][0]['message']['content']
                    return score
                
                score = chat_completion(prompt)
            except (ServiceUnavailableError, APIError, Timeout) as error:
                score = -1
            sents.append([text, score])
            
            sents = pd.DataFrame(sents, columns=['comment', 'sentiment'])
            
            if i==0:
                sents.to_csv(path + '/Support Files/sentiments.csv', mode='a', index=False, header=True)
            else:
                sents.to_csv(path + '/Support Files/sentiments.csv', mode='a', index=False, header=False)
            
            i += 1
            # time.sleep(1)
        
    beer_reviews['sentiment'] = beer_reviews[['final_rating', 'comment']].apply(tuple, axis=1)
    sentiment_openai(beer_reviews['sentiment'])
    beer_reviews = beer_reviews.drop(['sentiment'], axis=1)

    sentiment = pd.read_csv(path + '/Support Files/sentiments.csv')
    sentiment = sentiment.astype({'sentiment':float})
    sentiment = sentiment.drop_duplicates('comment', keep='last')
    beer_reviews = beer_reviews.merge(sentiment, left_on='comment', right_on='comment', how='left')
    display(beer_reviews)
else:
    sentiment = pd.read_csv(path + '/Support Files/sentiments.csv')
    sentiment = sentiment.astype({'sentiment':float})
    sentiment = sentiment.drop_duplicates('comment', keep='last')
    beer_reviews = beer_reviews.merge(sentiment, left_on='comment', right_on='comment', how='left')
    display(beer_reviews)

,pos,beer_name,final_rating,looks,smell,taste,feel,overall,user,place,...,tart_tf-idf,vintage_tf-idf,herbal_tf-idf,cherry_tf-idf,malty_tf-idf,dark_tf-idf,complex_tf-idf,diacetyl_tf-idf,similarity,sentiment
0,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,poidog,Hawaii,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,4.00
1,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,Filabee,Michigan,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,4.00
2,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,alexnblanton,Arkansas,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,-3.00
3,1,Kentucky Brunch Brand Stout,4.67,4.00,4.50,5.00,4.50,4.50,beergoot,Colorado,...,0.0,0.000000,0.0,0.000000,0.000000,1.996075,0.000000,0.0,0.00000,3.50
4,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,dcotom,Iowa,...,0.0,1.338099,0.0,1.541265,0.929784,0.000000,0.000000,0.0,0.00000,4.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11888,250,Samuel Adams Utopias,4.76,4.25,5.00,4.75,4.50,4.75,Robomunky,Washington,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,4.76
11889,250,Samuel Adams Utopias,4.49,3.75,4.75,4.50,4.25,4.50,chinchill,South Carolina,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.028406,0.0,0.39424,4.49
11890,250,Samuel Adams Utopias,4.38,4.50,4.00,4.50,4.50,4.50,superspak,North Carolina,...,0.0,1.338099,0.0,6.165061,0.929784,2.661433,4.113625,0.0,0.39424,3.89
11891,250,Samuel Adams Utopias,4.63,4.00,4.75,4.75,4.50,4.50,skiking70,Maryland,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,4.63


In [142]:
beer_reviews['final_score'] = beer_reviews['similarity'] * beer_reviews['sentiment']
beer_reviews

,pos,beer_name,final_rating,looks,smell,taste,feel,overall,user,place,...,vintage_tf-idf,herbal_tf-idf,cherry_tf-idf,malty_tf-idf,dark_tf-idf,complex_tf-idf,diacetyl_tf-idf,similarity,sentiment,final_score
0,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,poidog,Hawaii,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,4.00,0.000000
1,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,Filabee,Michigan,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,4.00,0.000000
2,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,alexnblanton,Arkansas,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,-3.00,-0.000000
3,1,Kentucky Brunch Brand Stout,4.67,4.00,4.50,5.00,4.50,4.50,beergoot,Colorado,...,0.000000,0.0,0.000000,0.000000,1.996075,0.000000,0.0,0.00000,3.50,0.000000
4,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,dcotom,Iowa,...,1.338099,0.0,1.541265,0.929784,0.000000,0.000000,0.0,0.00000,4.80,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11888,250,Samuel Adams Utopias,4.76,4.25,5.00,4.75,4.50,4.75,Robomunky,Washington,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,4.76,0.000000
11889,250,Samuel Adams Utopias,4.49,3.75,4.75,4.50,4.25,4.50,chinchill,South Carolina,...,0.000000,0.0,0.000000,0.000000,0.000000,1.028406,0.0,0.39424,4.49,1.770138
11890,250,Samuel Adams Utopias,4.38,4.50,4.00,4.50,4.50,4.50,superspak,North Carolina,...,1.338099,0.0,6.165061,0.929784,2.661433,4.113625,0.0,0.39424,3.89,1.533594
11891,250,Samuel Adams Utopias,4.63,4.00,4.75,4.75,4.50,4.50,skiking70,Maryland,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,4.63,0.000000


In [150]:
recommendations = list(beer_reviews.loc[beer_reviews['final_score'] > 0.5]['beer_name'].value_counts()[:3].index)
print('Three recommended beers:')
print(recommendations)

Three recommended beers:
['CBS (Canadian Breakfast Stout)', 'Bourbon County Brand Stout', 'KBS']


In [144]:
attributes = passer

re_calculate = False
if re_calculate:
    def spacySimilarity(comments, attributes, map):
        nlp = spacy.load("en_core_web_md")

        comments = list(comments)
        attributes = list(attributes)
        similarity_scores = []
        for comment in tqdm(comments):
            row = []
            row.append(comment)
            for attribute in attributes:
                text1 = attribute
                text2 = comment

                text2 = text2.strip()
                for punct in list(string.punctuation):
                    text2 = text2.replace(punct, ' ')
                text2 = text2.lower()

                l_text = text2.split()
                for word in l_text:
                    if word in map.keys():
                        text2 = text2.replace(word, map[word])

                text2 = ''.join(text2)

                doc1 = nlp(text1)
                doc2 = nlp(text2)
                similarity = doc1.similarity(doc2)
                row.append(similarity)
            similarity_scores.append(row)
        
        new_attributtes = ['comment']
        for i in range(len(attributes)):
            new_attributtes.append(attributes[i] + "_SSimilarity")
        
        similarity = pd.DataFrame(similarity_scores, columns=new_attributtes)
        similarity.to_csv(path + '/Support Files/similarity_spacy.csv', index=False)

    spacySimilarity(beer_reviews['comment'], set(beer_attribute_dict.values()), beer_attribute_dict)
    
    new_attributtes = ['comment']
    for i in range(len(attributes)):
        new_attributtes.append(attributes[i] + "_SSimilarity")
    
    similarity = pd.read_csv(path + '/Support Files/similarity_spacy.csv').drop_duplicates()
    similarity = similarity[new_attributtes]
    beer_reviews = beer_reviews.merge(similarity, left_on='comment', right_on='comment', how='left')
    display(beer_reviews)
else:
    new_attributtes = ['comment']
    for i in range(len(attributes)):
        new_attributtes.append(attributes[i] + "_SSimilarity")
    
    similarity = pd.read_csv(path + '/Support Files/similarity_spacy.csv').drop_duplicates()
    similarity = similarity[new_attributtes]
    beer_reviews = beer_reviews.merge(similarity, left_on='comment', right_on='comment', how='left')
    display(beer_reviews)

,pos,beer_name,final_rating,looks,smell,taste,feel,overall,user,place,...,malty_tf-idf,dark_tf-idf,complex_tf-idf,diacetyl_tf-idf,similarity,sentiment,final_score,complex_SSimilarity,aggressive_SSimilarity,silky_SSimilarity
0,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,poidog,Hawaii,...,0.000000,0.000000,0.000000,0.0,0.00000,4.00,0.000000,0.564415,0.561030,0.272089
1,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,Filabee,Michigan,...,0.000000,0.000000,0.000000,0.0,0.00000,4.00,0.000000,0.584772,0.537163,0.305450
2,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,alexnblanton,Arkansas,...,0.000000,0.000000,0.000000,0.0,0.00000,-3.00,-0.000000,0.494517,0.497814,0.314359
3,1,Kentucky Brunch Brand Stout,4.67,4.00,4.50,5.00,4.50,4.50,beergoot,Colorado,...,0.000000,1.996075,0.000000,0.0,0.00000,3.50,0.000000,0.630255,0.580946,0.388669
4,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,dcotom,Iowa,...,0.929784,0.000000,0.000000,0.0,0.00000,4.80,0.000000,0.672140,0.594534,0.373191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11888,250,Samuel Adams Utopias,4.76,4.25,5.00,4.75,4.50,4.75,Robomunky,Washington,...,0.000000,0.000000,0.000000,0.0,0.00000,4.76,0.000000,0.483684,0.510057,0.336017
11889,250,Samuel Adams Utopias,4.49,3.75,4.75,4.50,4.25,4.50,chinchill,South Carolina,...,0.000000,0.000000,1.028406,0.0,0.39424,4.49,1.770138,0.642689,0.568737,0.385848
11890,250,Samuel Adams Utopias,4.38,4.50,4.00,4.50,4.50,4.50,superspak,North Carolina,...,0.929784,2.661433,4.113625,0.0,0.39424,3.89,1.533594,0.602274,0.482689,0.577632
11891,250,Samuel Adams Utopias,4.63,4.00,4.75,4.75,4.50,4.50,skiking70,Maryland,...,0.000000,0.000000,0.000000,0.0,0.00000,4.63,0.000000,0.503437,0.482675,0.282564


In [145]:
new_attributtes = new_attributtes[1:]
beer_reviews['Ssimilarity'] = 1
for attri in new_attributtes:
    beer_reviews['Ssimilarity'] = beer_reviews['Ssimilarity'] * beer_reviews[attri]

beer_reviews['Sffinal_score'] = beer_reviews['Ssimilarity'] * beer_reviews['sentiment']
beer_reviews

,pos,beer_name,final_rating,looks,smell,taste,feel,overall,user,place,...,complex_tf-idf,diacetyl_tf-idf,similarity,sentiment,final_score,complex_SSimilarity,aggressive_SSimilarity,silky_SSimilarity,Ssimilarity,Sffinal_score
0,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,poidog,Hawaii,...,0.000000,0.0,0.00000,4.00,0.000000,0.564415,0.561030,0.272089,0.086158,0.344632
1,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,Filabee,Michigan,...,0.000000,0.0,0.00000,4.00,0.000000,0.584772,0.537163,0.305450,0.095947,0.383789
2,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,alexnblanton,Arkansas,...,0.000000,0.0,0.00000,-3.00,-0.000000,0.494517,0.497814,0.314359,0.077388,-0.232164
3,1,Kentucky Brunch Brand Stout,4.67,4.00,4.50,5.00,4.50,4.50,beergoot,Colorado,...,0.000000,0.0,0.00000,3.50,0.000000,0.630255,0.580946,0.388669,0.142309,0.498081
4,1,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,5.00,dcotom,Iowa,...,0.000000,0.0,0.00000,4.80,0.000000,0.672140,0.594534,0.373191,0.149131,0.715828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11888,250,Samuel Adams Utopias,4.76,4.25,5.00,4.75,4.50,4.75,Robomunky,Washington,...,0.000000,0.0,0.00000,4.76,0.000000,0.483684,0.510057,0.336017,0.082898,0.394593
11889,250,Samuel Adams Utopias,4.49,3.75,4.75,4.50,4.25,4.50,chinchill,South Carolina,...,1.028406,0.0,0.39424,4.49,1.770138,0.642689,0.568737,0.385848,0.141036,0.633250
11890,250,Samuel Adams Utopias,4.38,4.50,4.00,4.50,4.50,4.50,superspak,North Carolina,...,4.113625,0.0,0.39424,3.89,1.533594,0.602274,0.482689,0.577632,0.167924,0.653225
11891,250,Samuel Adams Utopias,4.63,4.00,4.75,4.75,4.50,4.50,skiking70,Maryland,...,0.000000,0.0,0.00000,4.63,0.000000,0.503437,0.482675,0.282564,0.068662,0.317905


In [149]:
recommendations = list(beer_reviews.loc[beer_reviews['Sffinal_score'] > 0.5]['beer_name'].value_counts()[:3].index)
print('Three recommended beers:')
print(recommendations)

Three recommended beers:
['King Sue', 'KBS - Maple Mackinac Fudge', 'CBS (Canadian Breakfast Stout)']
